<a href="https://colab.research.google.com/github/siddharth101/Double_suspension/blob/main/2_wire_single_block.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sympy import zeros, symbols
from sympy import Matrix
from sympy.physics.mechanics import Body, PinJoint, SphericalJoint, PrismaticJoint, JointsMethod, inertia
from sympy.physics.mechanics import dynamicsymbols
from sympy import Symbol
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame, outer
from sympy.physics.mechanics import ReferenceFrame, Point, RigidBody
from sympy.physics.mechanics import kinetic_energy, potential_energy, Point, Particle

import sympy as smp
import numpy as np
import matplotlib.pyplot as plt
import sympy.physics.mechanics as me

In [2]:
alpha_x, alpha_y, alpha_z = dynamicsymbols('alpha_x, alpha_y, alpha_z')
q1, q2, q3, q4, u1, u2, u3, u4 = dynamicsymbols('q1 q2 q3 q4 u1 u2 u3 u4')
alpha_x1, alpha_y1, alpha_z1 = dynamicsymbols('alpha_x1 alpha_y1, alpha_z1')
beta_x1, beta_y1, beta_z1 = dynamicsymbols('beta_x1 beta_y1 beta_z1')
gamma_x1, gamma_y1, gamma_z1 = dynamicsymbols('gamma_x1, gamma_y1, gamma_z1')
eta_x1, eta_y1, eta_z1 = dynamicsymbols('eta_x1, eta_y1, eta_z1')

omega_x1, omega_y1, omega_z1 = dynamicsymbols('omega_x1 omega_y1 omega_z1')
x_0, x_1 = dynamicsymbols('x_0 x_1')
L = symbols('L')
ixx1, iyy1, izz1 = symbols('ixx1 iyy1 izz1')
M1, t = symbols('M1 t')


N = ReferenceFrame('N') # global
N_s  = ReferenceFrame('N_s') # suspension point
N_p1 = ReferenceFrame('N_p1') # wire 1
N_p2 = ReferenceFrame('N_p2') # wire 2
N_b = ReferenceFrame('N_b') # body frame


N_s.orient_axis(N, 0, N.z)
N_b.orient_axis(N, 0, N.z)


q1, q2, q3, q4, q5, q6 = dynamicsymbols('q1, q2, q3, q4, q5, q6', real=True, positive=True)
u1, u2, u3, u4, u5, u6 = dynamicsymbols('u1, u2, u3, u4, u5, u6')
w1, w2, w3, d1, d2, d3 = symbols('w1 w2 w3 d1 d2 d3')
ixx1, iyy1, izz1 = symbols('ixx1 iyy1 izz1')
ixx2, iyy2, izz2 = symbols('ixx2 iyy2 izz2')
M1, M2 = symbols('M1 M2')

In [3]:
O = Point('O')
P = Point('P')

P1 = Point('P1')
P1a = Point('P1a')

P2 = Point('P2')
P2a = Point('P2a')

X1 = Point('X1')


B1 = Point('B1')
B2 = Point('B2')
B1a = Point('B1a')
B2a = Point('B2a')

In [4]:
P.set_pos(O, q1*N.x)
X1.set_pos(O, q1*N.x - q2*N.y) #COM

P1.set_pos(P, -w1*N_s.x) #Suspension points
P2.set_pos(P,  w1*N_s.x)

B1.set_pos(X1, -w2*N_b.x + d2*N_b.y) #attachment points
B2.set_pos(X1,  w2*N_b.x + d2*N_b.y)


In [5]:
B1.pos_from(O).express(N)

(-w2 + q1(t))*N.x + (d2 - q2(t))*N.y

In [6]:
B1.pos_from(P1).express(N)

(w1 - w2)*N.x + (d2 - q2(t))*N.y

In [7]:
X1.pos_from(P1).express(N)

w1*N.x - q2(t)*N.y

In [8]:
from sympy import atan2
angle_p1p1a = atan2(B1.pos_from(P1).express(N).dot(N.y), B1.pos_from(P1).express(N).dot(N.x))
angle_p2p2a = atan2(B2.pos_from(P2).express(N).dot(N.y), B2.pos_from(P2).express(N).dot(N.x))

In [9]:
angle_p1p1a

atan2(d2 - q2(t), w1 - w2)

In [10]:
angle_p2p2a

atan2(d2 - q2(t), -w1 + w2)

In [11]:
p1body = Body('p1body', masscenter=P1, frame=N_s)
p2body = Body('p2body', masscenter=P2, frame=N_s)

p1abody = Body('p1abody', masscenter=P1a, frame=N_p1)
p2abody = Body('p2abody', masscenter=P2a, frame=N_p2)

In [12]:
## Spherical Joint
revp1 = SphericalJoint(name='p1p1a', parent=p1body, child=p1abody, parent_point=P1, child_point=P1a, amounts=[0,0, angle_p1p1a],
                     rot_order='XYZ')
revp2 = SphericalJoint(name='p2p2a', parent=p2body, child=p2abody, parent_point=P2, child_point=P2a, amounts=[0,0,angle_p2p2a],
                     rot_order='XYZ')

In [14]:
revp1.parent_interframe, revp1.child_interframe

(N_s, N_p1)

In [15]:
revp1.child_interframe.dcm(revp1.parent_interframe)

Matrix([
[    (w1 - w2)/sqrt((d2 - q2(t))**2 + (w1 - w2)**2), (d2 - q2(t))/sqrt((d2 - q2(t))**2 + (w1 - w2)**2), 0],
[-(d2 - q2(t))/sqrt((d2 - q2(t))**2 + (w1 - w2)**2),    (w1 - w2)/sqrt((d2 - q2(t))**2 + (w1 - w2)**2), 0],
[                                                 0,                                                 0, 1]])

In [16]:
revp2.child_interframe.dcm(revp2.parent_interframe)

Matrix([
[   (-w1 + w2)/sqrt((d2 - q2(t))**2 + (-w1 + w2)**2), (d2 - q2(t))/sqrt((d2 - q2(t))**2 + (-w1 + w2)**2), 0],
[-(d2 - q2(t))/sqrt((d2 - q2(t))**2 + (-w1 + w2)**2),   (-w1 + w2)/sqrt((d2 - q2(t))**2 + (-w1 + w2)**2), 0],
[                                                  0,                                                  0, 1]])

In [17]:
b1body = Body('b1body', masscenter=B1, frame=N_b)
b2body = Body('b2body', masscenter=B1, frame=N_b)

In [18]:
revp1ab1 = PrismaticJoint('p1ab1', parent=p1abody, child=b1body, coordinates=q3, parent_point=P1a, child_point=B1,
                      joint_axis = -N_p1.y, speeds=u3)
revp2ab2 = PrismaticJoint('p2ab2', parent=p2abody, child=b2body, coordinates=q3, parent_point=P2a, child_point=B2,
                      joint_axis = -N_p2.y, speeds=u3)

/usr/local/lib/python3.10/dist-packages/sympy/physics/vector/frame.py:580: UserWarning: Loops are defined among the orientation of frames. This is likely not desired and may cause errors in your calculations.
  warn('Loops are defined among the orientation of '


In [19]:
B1.pos_from(X1), B2.pos_from(X1) # still the same, which is good

(- w2*N_b.x + d2*N_b.y, w2*N_b.x + d2*N_b.y)

In [24]:
B1.pos_from(O).express(N) # this hasn't changed

(-w2 + q1(t))*N.x + (d2 - q2(t))*N.y

In [23]:
B1.pos_from(P1)

- q3(t)*N_p1.y

In [21]:
B1.pos_from(P1).express(N)

(d2 - q2(t))*q3(t)/sqrt((d2 - q2(t))**2 + (w1 - w2)**2)*N.x - (w1 - w2)*q3(t)/sqrt((d2 - q2(t))**2 + (w1 - w2)**2)*N.y